# NLP assignment 2 Group 4

## Members

1. Atharva Saney (23PGAI0069)
2. Mayank Khurana (23PGAI0004)
3. Pushkar Pushp (23PGAI0061)
4. Tanuja Tanushree (23PGAI0104)

## Important instructions

1. Run all the cells one by one (except Additional data)
2. While asking questions, end questions with '?'
3. Shortly after asking the questions, sometimes you may get an error saying 'Exception in Tkinter callback' of 'List index out of range. Please ignore that, your app is still running perfectly, continue your operations as usual.  
4. Please ignore the **Additional data** section, it is for testing and future scope.


## Import Libraries

In [1]:
from tkinter import *
from PIL import Image, ImageTk
import time
import subprocess
import wolframalpha
import pyttsx3
import json
import random
import operator
import speech_recognition as sr
import datetime
import wikipedia
import webbrowser
import os
import winshell
import pyjokes
import feedparser
import smtplib
import ctypes
import requests
import shutil
from twilio.rest import Client
from clint.textui import progress
from ecapture import ecapture as ec
from bs4 import BeautifulSoup
import win32com.client as wincl
from urllib.request import urlopen

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import pandas as pd
from transformers import pipeline
from keras import *


# defaults

defFont = ("Bookman Old Style", 12)

engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

website = 'https://recipes.timesofindia.com/recipes/'
req_attrib = ['quantity','unit','suffix','displayname']

qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
All model checkpoint layers were used when initializing TFDistilBertForQuestionAnswering.

All the layers of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


## Helper functions

In [2]:
def get_recipe_toi(website,recipe,req_attrib):
    driver = webdriver.Chrome()
    ingred = dict()
    steps = dict()
    driver.get(website)
    if driver.find_elements(By.XPATH,"//span[@class = 'ntfc_buttongroup']") != []:
        driver.find_element(By.XPATH,"//span[@class = 'ntfc_buttongroup']").click()
    driver.find_element(By.XPATH,"//input[@placeholder = 'Find a recipe...']").click()
    driver.find_element(By.XPATH,"//input[@placeholder = 'Find a recipe...']").send_keys(recipe)
    time.sleep(5)
    driver.find_element(By.XPATH,"//div[@id = 'searchListing']//a").click()
    time.sleep(3)
    #try:
    #    driver.find_elements(By.XPATH,"//div[@class = 'servingselect']//option")[serving-1].click()
    #except IndexError as E:
    #    # todo : handle outofbound serving case by multiplying by single proportion.
    #    driver.find_elements(By.XPATH,"//div[@class = 'servingselect']//option")[1].click()

    temp = 0
    for i in driver.find_elements(By.XPATH,"//div[@id = 'ingredata']//li[@class='clearfix']"):
        ingred[temp] = dict()
        for j in req_attrib:
            ingred[temp][j] = i.get_attribute(j)
            #print(j+': '+i.get_attribute(j))
        #ingred
        temp+=1
   
    tempIndex = 0
    
    for i in range(len(driver.find_elements(By.XPATH,"//div[@class = 'steps_listings clearfix']//p"))):
        sentences = driver.find_elements(By.XPATH,"//div[@class = 'steps_listings clearfix']//p")[i].text.split('.')
        for j in sentences:
            if j != '':
                steps[tempIndex] = j
                tempIndex += 1
            
    #print('step',i)
    #print(driver.find_elements(By.XPATH,"//div[@class = 'steps_listings clearfix']//p")[i].text)
    driver.close()
    return ingred,steps


def questionAnswerModel(question):
    context = 'Ingredients: ' + displayIngredients.replace('\n', ', ') + '. Steps: ' + str(allSteps)
    
    # types of questions
    alternate = re.search('can i use|alternatively|alternate|in place of|replace|substitute|alternative|instead of ', 
                          question, re.IGNORECASE)
    howTo = re.search('how to|how do i |what is the procedure|how does one ', question, re.IGNORECASE)
    what = re.search('what ', question, re.IGNORECASE)
    googleWhat = re.search('what is|what do you mean by |what is the meaning of', question, re.IGNORECASE)
    stdQ1 = re.search('what are the ingredients|list of ingredients|go back to ingredients', question, re.IGNORECASE)
    
    if stdQ1:
        return displayIngredients
    
    if howTo:
        driver = webdriver.Chrome()
        driver.get('http://www.youtube.com')
        driver.maximize_window()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//input[@id="search"]')))
        driver.find_element(By.XPATH,'//input[@id="search"]').click()
        search = driver.find_element(By.XPATH,'//input[@id="search"]')
        search.send_keys(question + Keys.RETURN)
        
        DISCONNECTED_MSG = 'Unable to evaluate script: disconnected: not connected to DevTools\n'
        
        while True:
            if driver.get_log('driver')[-1]['message'] == DISCONNECTED_MSG:
                return
            else:
                time.sleep(1)
    
    if (what and alternate) or googleWhat:
        driver = webdriver.Chrome()
        driver.get('https://www.google.com/')
        driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").click()
        driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(
            question + Keys.RETURN)
        DISCONNECTED_MSG = 'Unable to evaluate script: disconnected: not connected to DevTools\n'
        
        while True:
            if driver.get_log('driver')[-1]['message'] == DISCONNECTED_MSG:
                return
            else:
                time.sleep(1)
    
    if alternate:
        return 'No'
    return qa_model(question = question, context = context)['answer']

def speak(audio):
    engine.say(audio)
    engine.runAndWait()

def wishMe():
    hour = int(datetime.datetime.now().hour)
    if hour>= 0 and hour<12:
        speak("Good Morning !")

    elif hour>= 12 and hour<18:
        speak("Good Afternoon !")

    else:
        speak("Good Evening !")

    assname =("Jarvis 1 point o")
    speak("I am your Assistant")
    speak(assname)


def username():
    speak("What should i call you ?")
    uname = takeCommand()
    speak("Welcome ")
    speak(uname)
    columns = shutil.get_terminal_size().columns

    print("#####################".center(columns))
    print("Welcome ", uname.center(columns))
    print("#####################".center(columns))

    speak("Let me open you coooking app display")

def takeCommand():
    r = sr.Recognizer()

    with sr.Microphone() as source:

        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)

    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language ='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        print(e)
        print("Unable to Recognize your voice.")
        return "None"

    return query

def sendEmail(to, content):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()

    # Enable low security in gmail
    server.login('your email id', 'your email password')
    server.sendmail('your email id', to, content)
    server.close()

## GUI logic

In [4]:
# what is the alternative to oven?

cooking_app_root = Tk()

# gui logic

# width x height
cooking_app_root.geometry('800x800')
cooking_app_root.title('Atharva Cooking App')

# important label options
# text - adds text, bd - background, fg - foreground, font - sets font, padx - padding x, pady
# relief - border styling, SUNKEN, RAISED, GROOVE, RIDGE

frames = []

titleFrame = Frame(cooking_app_root, borderwidth = 8, bg = 'black')
titleFrame.pack(side = 'top', fill = X)

Label(titleFrame, text = "Welcome to Atharva's cooking App !!", 
      font = defFont, bg = 'black', fg = 'white', relief = 'solid').pack(anchor = 'center')


def homePage():
    for i in frames:
        try:
            i.pack_forget()
        except:
            pass
        
    global homeFrame
    homeFrame = Frame(cooking_app_root, borderwidth = 8, bg = 'white', width = 800)
    frames.append(homeFrame)
    homeFrame.pack(side = 'top', fill = BOTH)

    Label(homeFrame, text = 'Enter the recipe you want', font = defFont, bg = 'white').pack()
    
    Label(homeFrame, text = '\n\n', font = defFont, bg = 'white').pack()
    
    global recipeEntry 
    recipeEntry = Entry(homeFrame, textvariable = StringVar(), font = defFont, relief = 'sunken')
    recipeEntry.pack(anchor = 'center')

    Button(homeFrame, text = 'Get Recipe', font = defFont, command = loadingPage).pack(anchor = 'center')
    
def loadingPage():
    for i in frames:
        try:
            i.pack_forget()
        except:
            pass
    
    global displayIngredients, ingred, steps
    
    displayIngredients = ""
        
    global loadFrame
    loadFrame = Frame(cooking_app_root, borderwidth = 8, bg = 'white', width = 800)
    frames.append(loadFrame)
    loadFrame.pack(side = 'top', fill = X)
    
    Label(loadFrame, text = 'Getting recipe for ' + recipeEntry.get() + '\nThis might take about 2 mins...', 
          font = defFont, bg = 'white').pack()
    
    global allSteps, ingred, steps
    allSteps = ''
    ingred, steps = get_recipe_toi(website,recipeEntry.get(),req_attrib)
    
    for i in range(len(ingred)):
        displayIngredients = displayIngredients + ingred[i]['quantity'] + " " + ingred[
        i]['unit'] + " " + ingred[i]['suffix'] + " " + ingred[i]['displayname'] + "\n"
    
    for i in range(len(steps)):
        allSteps = allSteps + '.' + steps[i]
    
    ingredientsPage()

def ingredientsPage():

    for i in frames:
        try:
            i.pack_forget()
        except:
            pass

    global ingFrame
    ingFrame = Frame(cooking_app_root, borderwidth = 8, bg = 'white', width = 800)
    frames.append(ingFrame)
    ingFrame.pack(side = 'top', fill = X)
    
    scrollbar = Scrollbar(ingFrame)
    scrollbar.pack(side="right",fill= Y)
    
    Label(ingFrame, text = 'These are the ingredients for ' + recipeEntry.get() + 
          '\nPlease ensure that you have all of them ready.\n then click Ingredients Ready\n\n', 
          font = defFont, bg = 'white').pack()

    mainText = Text(ingFrame, wrap = WORD, font = defFont)
    mainText.insert(INSERT, displayIngredients)
    mainText.tag_configure("tag_name", justify='center')
    mainText.insert("1.0", "")
    mainText.tag_add("tag_name", "1.0", "end")
    mainText.pack()


    Button(ingFrame, text = 'Exit to Home Page', font = defFont, command = homePage).pack(anchor = 'center')
    
    Button(ingFrame, text = 'Ingredients Ready', font = defFont, command = recipePage).pack(side = 'right')

def recipePage():
    
    ingFrame.pack_forget()
    
    def buttonsWithCommands(frame, step, stepNo):
        Label(frame, text = '\n\n', font = defFont, bg = 'white').pack()

        Button(frame, text = 'Next Step', font = defFont, command = lambda: nextStep(step, stepNo)).pack(
            side = 'right')

        Button(frame, text = 'Previous Step', font = defFont, command = lambda: prevStep(step, stepNo)).pack(
            side = 'left')

        Button(frame, text = 'Exit to Home Page', font = defFont, command = homePage).pack(
            anchor = 'center', side = 'bottom')

        Label(frame, text = '\n\n', font = defFont, bg = 'white').pack()

        Button(frame, text= "Ask a Question", font = defFont, command= lambda: open_popup(frame)).pack(anchor = 'center')

        Label(frame, text = '\n\n', font = defFont, bg = 'white').pack()

    def open_popup(frame):
        top = Toplevel(frame)
        top.geometry("800x800")
        top.title("Help Center")
        Label(top, text = 'Please type your question', font = defFont).pack()
        Label(top, text = '\n\n', font = defFont).pack()
        userQuestion = Entry(top, textvariable = StringVar(), font = defFont, relief = 'sunken', width = 50)
        userQuestion.pack()
        displayAnswer = Label(top, text = '', font = defFont)
        askButton = Button(top, text = 'Ask', font = defFont, 
                           command = lambda: askQuestion(userQuestion.get(), top, displayAnswer)).pack()

    def askQuestion(question, top, displayAnswer):
        userAnswer = questionAnswerModel(question)
        Label(top, text = '\n\n', font = defFont).pack()
        displayAnswer = Label(top, text = userAnswer, font = defFont)
        displayAnswer.pack()
        
        clearAnswer = Button(top, text = 'Clear Answer', font = defFont, 
                           command = lambda: destroyPopupAnswer(displayAnswer, clearAnswer))
        clearAnswer.pack()
           
    def destroyPopupAnswer(displayAnswer, clearAnswer):
        displayAnswer.destroy()
        clearAnswer.destroy()
        

    stepNo = 0
    
    global recipeFrame
    recipeFrame = Frame(cooking_app_root, borderwidth = 8, bg = 'white', width = 800)
    frames.append(recipeFrame)
    recipeFrame.pack(side = 'top', fill = X)
    
    def nextStep(currentStep, stepNo):
        
        currentStep.destroy()
        for i in recipeFrame.pack_slaves():
            i.destroy()
        
        Label(recipeFrame, text = 'INSTRUCTIONS\n\n', font = defFont, bg = 'white').pack()
        
        if stepNo < len(steps) - 1:
            stepNo = stepNo + 1
            currentStep = Label(recipeFrame, text = str(stepNo + 1) + '. ' + steps[stepNo], font = defFont, bg = 'white')
            currentStep.bind('<Configure>', lambda e: currentStep.config(wraplength=currentStep.winfo_width()))
            currentStep.pack()
        
        else:
            currentStep = Label(recipeFrame, text = str(recipeEntry.get()) + " is ready, Enjoy your meal !!", 
                                font = defFont, bg = 'white')
            currentStep.bind('<Configure>', lambda e: currentStep.config(wraplength=currentStep.winfo_width()))
            currentStep.pack()
        
        buttonsWithCommands(recipeFrame, currentStep, stepNo)
    
    def prevStep(currentStep, stepNo):
        
        currentStep.destroy()
        for i in recipeFrame.pack_slaves():
            i.destroy()
        
        if stepNo > 0:
            stepNo = stepNo - 1
        else:
            pass
        
        Label(recipeFrame, text = 'INSTRUCTIONS\n\n', font = defFont, bg = 'white').pack()
        
        currentStep = Label(recipeFrame, text = str(stepNo + 1) + '. ' + steps[stepNo], font = defFont, bg = 'white')
        currentStep.bind('<Configure>', lambda e: currentStep.config(wraplength=currentStep.winfo_width()))
        currentStep.pack()
        
        buttonsWithCommands(recipeFrame, currentStep, stepNo)
            
    Label(recipeFrame, text = 'INSTRUCTIONS\n\n', font = defFont, bg = 'white').pack()
    
    currentStep = Label(recipeFrame, text = str(stepNo + 1) + '. ' + steps[stepNo], font = defFont, bg = 'white')
    currentStep.bind('<Configure>', lambda e: currentStep.config(wraplength=currentStep.winfo_width()))
    currentStep.pack()
    
    buttonsWithCommands(recipeFrame, currentStep, stepNo)
    
homePage()

cooking_app_root.mainloop()

## Additional Data (Please ignore)

In [10]:
clear = lambda: os.system('cls')

# This Function will clean any
# command before execution of this python file
clear()
wishMe()
username()

Listening...
Recognizing...
User said: Atharvaa

                                                 #####################                                                  
Welcome                                                          Atharvaa                                                        
                                                 #####################                                                  


In [ ]:
while True:
		
		query = takeCommand().lower()
		
		# All the commands said by user will be
		# stored here in 'query' and will be
		# converted to lower case for easily
		# recognition of command
		if 'wikipedia' in query:
			speak('Searching Wikipedia...')
			query = query.replace("wikipedia", "")
			results = wikipedia.summary(query, sentences = 3)
			speak("According to Wikipedia")
			print(results)
			speak(results)

		elif 'open youtube' in query:
			speak("Here you go to Youtube\n")
			webbrowser.open("youtube.com")

		elif 'open google' in query:
			speak("Here you go to Google\n")
			webbrowser.open("google.com")

		elif 'open stackoverflow' in query:
			speak("Here you go to Stack Over flow.Happy coding")
			webbrowser.open("stackoverflow.com")

		elif 'play music' in query or "play song" in query:
			speak("Here you go with music")
			# music_dir = "G:\\Song"
			music_dir = "C:\\Users\\GAURAV\\Music"
			songs = os.listdir(music_dir)
			print(songs)
			random = os.startfile(os.path.join(music_dir, songs[1]))

		elif 'the time' in query:
			strTime = datetime.datetime.now().strftime("% H:% M:% S")
			speak(f"Sir, the time is {strTime}")

		elif 'open opera' in query:
			codePath = r"C:\\Users\\GAURAV\\AppData\\Local\\Programs\\Opera\\launcher.exe"
			os.startfile(codePath)

		elif 'email to gaurav' in query:
			try:
				speak("What should I say?")
				content = takeCommand()
				to = "Receiver email address"
				sendEmail(to, content)
				speak("Email has been sent !")
			except Exception as e:
				print(e)
				speak("I am not able to send this email")

		elif 'send a mail' in query:
			try:
				speak("What should I say?")
				content = takeCommand()
				speak("whome should i send")
				to = input()
				sendEmail(to, content)
				speak("Email has been sent !")
			except Exception as e:
				print(e)
				speak("I am not able to send this email")

		elif 'how are you' in query:
			speak("I am fine, Thank you")
			speak("How are you, Sir")

		elif 'fine' in query or "good" in query:
			speak("It's good to know that your fine")

		elif "change my name to" in query:
			query = query.replace("change my name to", "")
			assname = query

		elif "change name" in query:
			speak("What would you like to call me, Sir ")
			assname = takeCommand()
			speak("Thanks for naming me")

		elif "what's your name" in query or "What is your name" in query:
			speak("My friends call me")
			speak(assname)
			print("My friends call me", assname)

		elif 'exit' in query:
			speak("Thanks for giving me your time")
			exit()

		elif "who made you" in query or "who created you" in query:
			speak("I have been created by Atharva Saney. He is the greatest coder in the world.")
			
		elif 'joke' in query:
			speak(pyjokes.get_joke())
			
		elif "calculate" in query:
			
			app_id = "Wolframalpha api id"
			client = wolframalpha.Client(app_id)
			indx = query.lower().split().index('calculate')
			query = query.split()[indx + 1:]
			res = client.query(' '.join(query))
			answer = next(res.results).text
			print("The answer is " + answer)
			speak("The answer is " + answer)

		elif 'search' in query or 'play' in query:
			
			query = query.replace("search", "")
			query = query.replace("play", "")		
			webbrowser.open(query)

		elif "who am i" in query:
			speak("If you talk then definitely your human.")

		elif "why do you exist" in query:
			speak("I exist to serve you")

		elif 'power point presentation' in query:
			speak("opening Power Point presentation")
			power = r"C:\\Users\\GAURAV\\Desktop\\Minor Project\\Presentation\\Voice Assistant.pptx"
			os.startfile(power)

		elif 'is love' in query:
			speak("It is 7th sense that destroy all other senses")

		elif "who are you" in query:
			speak("I am your virtual assistant created by Atharva Saney the great")

		elif 'reason for you' in query:
			speak("I was created as a servent project by Mister Atharva Saney the great")

		elif 'change background' in query:
			ctypes.windll.user32.SystemParametersInfoW(20,
													0,
													"Location of wallpaper",
													0)
			speak("Background changed successfully")

		elif 'open bluestack' in query:
			appli = r"C:\\ProgramData\\BlueStacks\\Client\\Bluestacks.exe"
			os.startfile(appli)

		elif 'news' in query:
			
			try:
				jsonObj = urlopen(
                    '''https://newsapi.org / v1 / articles?source = the-times-of-india&s
                    ortBy = top&apiKey =\\times of India Api key\\''')
				data = json.load(jsonObj)
				i = 1
				
				speak('here are some top news from the times of india')
				print('''=============== TIMES OF INDIA ============'''+ '\n')
				
				for item in data['articles']:
					
					print(str(i) + '. ' + item['title'] + '\n')
					print(item['description'] + '\n')
					speak(str(i) + '. ' + item['title'] + '\n')
					i += 1
			except Exception as e:
				
				print(str(e))

		
		elif 'lock window' in query:
				speak("locking the device")
				ctypes.windll.user32.LockWorkStation()

		elif 'shutdown system' in query:
				speak("Hold On a Sec ! Your system is on its way to shut down")
				subprocess.call('shutdown / p /f')
				
		elif 'empty recycle bin' in query:
			winshell.recycle_bin().empty(confirm = False, show_progress = False, sound = True)
			speak("Recycle Bin Recycled")

		elif "don't listen" in query or "stop listening" in query:
			speak("for how much time you want to stop jarvis from listening commands")
			a = int(takeCommand())
			time.sleep(a)
			print(a)

		elif "where is" in query:
			query = query.replace("where is", "")
			location = query
			speak("User asked to Locate")
			speak(location)
			webbrowser.open("https://www.google.nl / maps / place/" + location + "")

		elif "camera" in query or "take a photo" in query:
			ec.capture(0, "Jarvis Camera ", "img.jpg")

		elif "restart" in query:
			subprocess.call(["shutdown", "/r"])
			
		elif "hibernate" in query or "sleep" in query:
			speak("Hibernating")
			subprocess.call("shutdown / h")

		elif "log off" in query or "sign out" in query:
			speak("Make sure all the application are closed before sign-out")
			time.sleep(5)
			subprocess.call(["shutdown", "/l"])

		elif "write a note" in query:
			speak("What should i write, sir")
			note = takeCommand()
			file = open('jarvis.txt', 'w')
			speak("Sir, Should i include date and time")
			snfm = takeCommand()
			if 'yes' in snfm or 'sure' in snfm:
				strTime = datetime.datetime.now().strftime("% H:% M:% S")
				file.write(strTime)
				file.write(" :- ")
				file.write(note)
			else:
				file.write(note)
		
		elif "show note" in query:
			speak("Showing Notes")
			file = open("jarvis.txt", "r")
			print(file.read())
			speak(file.read(6))

		elif "update assistant" in query:
			speak("After downloading file please replace this file with the downloaded one")
			url = '# url after uploading file'
			r = requests.get(url, stream = True)
			
			with open("Voice.py", "wb") as Pypdf:
				
				total_length = int(r.headers.get('content-length'))
				
				for ch in progress.bar(r.iter_content(chunk_size = 2391975),
									expected_size =(total_length / 1024) + 1):
					if ch:
						Pypdf.write(ch)
					
		# NPPR9-FWDCX-D2C8J-H872K-2YT43
		elif "jarvis" in query:
			
			wishMe()
			speak("Jarvis 1 point o in your service Mister")

		elif "weather" in query:
			
			# Google Open weather website
			# to get API of Open weather
			api_key = "Api key"
			base_url = "http://api.openweathermap.org / data / 2.5 / weather?"
			speak(" City name ")
			print("City name : ")
			city_name = takeCommand()
			complete_url = base_url + "appid =" + api_key + "&q =" + city_name
			response = requests.get(complete_url)
			x = response.json()
			
			if x["code"] != "404":
				y = x["main"]
				current_temperature = y["temp"]
				current_pressure = y["pressure"]
				current_humidiy = y["humidity"]
				z = x["weather"]
				weather_description = z[0]["description"]
				print(" Temperature (in kelvin unit) = " +str(
                    current_temperature)+"\n atmospheric pressure (in hPa unit) ="+str(
                    current_pressure) +"\n humidity (in percentage) = " +str(
                    current_humidiy) +"\n description = " +str(weather_description))
			
			else:
				speak(" City Not Found ")
			
		elif "send message " in query:
				# You need to create an account on Twilio to use this service
				account_sid = 'Account Sid key'
				auth_token = 'Auth token'
				client = Client(account_sid, auth_token)

				message = client.messages \
								.create(
									body = takeCommand(),
									from_='Sender No',
									to ='Receiver No'
								)

				print(message.sid)

		elif "wikipedia" in query:
			webbrowser.open("wikipedia.com")

		elif "Good Morning" in query:
			speak("A warm" +query)
			speak("How are you Mister")
			speak(assname)

		# most asked question from google Assistant
		elif "will you be my gf" in query or "will you be my bf" in query:
			speak("I'm not sure about, may be you should give me some time")

		elif "how are you" in query:
			speak("I'm fine, glad you me that")

		elif "i love you" in query:
			speak("It's hard to understand")

		elif "what is" in query or "who is" in query:
			
			# Use the same API key
			# that we have generated earlier
			client = wolframalpha.Client("API_ID")
			res = client.query(query)
			
			try:
				print (next(res.results).text)
				speak (next(res.results).text)
			except StopIteration:
				print ("No results")

		# elif "" in query:
			# Command go here
			# For adding more commands


In [65]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

txt = 'What is the unit of schezwan sauce?'

tokenized = sent_tokenize(txt)
for i in tokenized:
     
    # Word tokenizers is used to find the words
    # and punctuation in a string
    wordsList = nltk.word_tokenize(i)
 
    #  Using a Tagger. Which is part-of-speech
    # tagger or POS-tagger.
    tagged = nltk.pos_tag(wordsList)
 
    print(tagged)

[('What', 'WP'), ('is', 'VBZ'), ('the', 'DT'), ('unit', 'NN'), ('of', 'IN'), ('schezwan', 'NN'), ('sauce', 'NN'), ('?', '.')]
